# Resolvendo Tarefas de Maneira Zero e Few-shot

Autora: Monique Monteiro

Neste notebook, é demonstrado o uso do modelo/API do ChatGPT (GPT 3.5) para uma tarefa de reconhecimento de entidades nomeadas (NER) em um dataset de notícias da mídia.  Versões anteriores desse mesmo dataset foram utilizados pela autora para treinamento com finetuning do BERTimbau na tarefa NER, tendo obtido as seguintes métricas:

* precision: 0.86 
* recall: 0.91 
* F1: 0.88

Referência: https://huggingface.co/monilouise/ner_news_portuguese



In [ ]:
pip install openai

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
main_path = "/content/gdrive/MyDrive/Unicamp-aula-4"

In [ ]:
max_length = 500

In [ ]:
entity_types_map = {"PESSOA":"PESSOA", "LOC":"LOCAL", "ORG":"ORGANIZAÇÃO PRIVADA", "PUB":"INSTITUIÇÃO PÚBLICA"} 

In [ ]:
import json
from collections import defaultdict

full_data = []
i = 0

with open(f"{main_path}/labeled_4_labels_pub.jsonl", 'r') as f:
    for line in f:
        data = json.loads(line)
        #Como os textos a serem repassados ao ChatGPT tên um limite de tamanho, tenho que criar um novo alvo (target) que contenha menos entidades
        reduced_text = ' '.join(data["text"].split()[0:max_length])
        data["reduced_text"] = reduced_text
        new_target = defaultdict(list)
        for start, end, ent_type in data["labels"]:
          if end < len(reduced_text):
            new_target[entity_types_map[ent_type]].append(data["text"][start:end].strip())
        new_data = {"text":reduced_text, "labels":new_target}
        full_data.append(new_data)
        i += 1


In [ ]:
print(full_data[68]['labels'])

defaultdict(<class 'list'>, {'PESSOA': ['Helder Barbalho', 'Hana Ghassan', 'Alberto Beltrame', 'Helder Barbalho', 'Alberto Beltrame', 'Hana Ghassan', 'Helder Barbalho', 'Helder Barbalho', 'Helder Barbalho', 'Alberto Beltrame', 'Alberto Beltrame'], 'INSTITUIÇÃO PÚBLICA': ['Governo do Pará', 'Secretaria de Saúde do Estado', 'Auditoria-Geral do Estado', 'AGE', 'Ministério Público Federal', 'MPF', 'Ministério Público do Estado', 'MP-PA', 'Ministério Público de Contas', 'MPC', 'Tribunal de Contas do Estado', 'TCE', 'Secretaria de Estado de Saúde Pública', 'Sespa']})


## Few-shot learning

In [ ]:
from collections import defaultdict

def convert_doccano_to_gpt(doc):
  text = ' '.join(doc["text"].split()[:max_length])
  len_text = len(text)
  text = "Sentença: " + text + ".\n Saída: "

  print(doc["labels"])
  #Ordena as entidades por posição
  #doc["labels"] = sorted(doc["labels"], key=lambda x: x[0])

  
  return text + str(doc["labels"]).replace("defaultdict(<class 'list'>,","")



In [ ]:
import random

random.seed(123)

few_shot_indices = random.sample(range(len(full_data)), 2)
few_shot_list = [full_data[i] for i in few_shot_indices]

In [ ]:
few_shot_indices

[13, 68]

In [ ]:
full_data[13]

{'text': 'TCE-RJ aponta sobrepreço na compra de respiradores Data: 25/05/2020 A auditoria foi realizada entre 27 e 30 de abril em três contratos assinados para a aquisição de mil respiradores, com valor total de R$ 183,5 milhões - Imprensa/RJ O Tribunal de Contas do Estado do Rio de Janeiro (TCE-RJ), por meio da Secretaria Geral de Controle Externo (SGE), concluiu o relatório de Auditoria Governamental para verificar irregularidades nos contratos emergenciais da Secretaria Estadual de Saúde do Rio para a aquisição de ventiladores pulmonares no combate à pandemia do novo coronavírus e apurou um sobrepreço de R$ 123 milhões na aquisição dos respiradores. A auditoria aponta o ex-secretário estadual de Saúde, Edmar José dos Santos e o ex-subsecretário Gabriell Carvalho Neves Franco dos Santos como principais responsáveis pelas irregularidades e solicita a apresentação de razões de defesa ou o ressarcimento aos cofres públicos de R$ 36,5 milhões. A auditoria foi realizada entre 27 e 30 de a

In [ ]:
few_shot_list

[{'text': 'TCE-RJ aponta sobrepreço na compra de respiradores Data: 25/05/2020 A auditoria foi realizada entre 27 e 30 de abril em três contratos assinados para a aquisição de mil respiradores, com valor total de R$ 183,5 milhões - Imprensa/RJ O Tribunal de Contas do Estado do Rio de Janeiro (TCE-RJ), por meio da Secretaria Geral de Controle Externo (SGE), concluiu o relatório de Auditoria Governamental para verificar irregularidades nos contratos emergenciais da Secretaria Estadual de Saúde do Rio para a aquisição de ventiladores pulmonares no combate à pandemia do novo coronavírus e apurou um sobrepreço de R$ 123 milhões na aquisição dos respiradores. A auditoria aponta o ex-secretário estadual de Saúde, Edmar José dos Santos e o ex-subsecretário Gabriell Carvalho Neves Franco dos Santos como principais responsáveis pelas irregularidades e solicita a apresentação de razões de defesa ou o ressarcimento aos cofres públicos de R$ 36,5 milhões. A auditoria foi realizada entre 27 e 30 de 

In [ ]:
few_shot_list = [convert_doccano_to_gpt(e) for e in few_shot_list]

defaultdict(<class 'list'>, {'PESSOA': ['Edmar José dos Santos', 'Gabriell Carvalho Neves Franco dos Santos'], 'ORGANIZAÇÃO PRIVADA': ['ARC Fontoura Indústria Comércio e Representações', 'A2A Comércio Serviços e Representações', 'MHS Produtos e Serviços', 'ARC Fontoura', 'A2A', 'MHS'], 'INSTITUIÇÃO PÚBLICA': ['TCE-RJ', 'Tribunal de Contas do Estado do Rio de Janeiro', 'TCE-RJ', 'Secretaria Geral de Controle Externo', 'SGE', 'Secretaria Estadual de Saúde do Rio', 'Secretaria Geral de Controle Externo do TCE-RJ', 'TCE-RJ', 'Ministério Público Estadual'], 'LOCAL': ['RJ']})
defaultdict(<class 'list'>, {'PESSOA': ['Helder Barbalho', 'Hana Ghassan', 'Alberto Beltrame', 'Helder Barbalho', 'Alberto Beltrame', 'Hana Ghassan', 'Helder Barbalho', 'Helder Barbalho', 'Helder Barbalho', 'Alberto Beltrame', 'Alberto Beltrame'], 'INSTITUIÇÃO PÚBLICA': ['Governo do Pará', 'Secretaria de Saúde do Estado', 'Auditoria-Geral do Estado', 'AGE', 'Ministério Público Federal', 'MPF', 'Ministério Público do Est

In [384]:
HEADER_PROMPT = "Diga quais entidades são do tipo PESSOA, do tipo INSTITUIÇÃO PÚBLICA, do tipo ORGANIZAÇÃO PRIVADA ou do tipo LOCAL."
guidelines_prompt = "Exemplos:\n"
for i, example in enumerate(few_shot_list):
  guidelines_prompt += f"{i+1}. {example}\n"
guidelines_prompt += "Sentença: "

In [ ]:
guidelines_prompt

'Exemplos:\n1. Sentença: TCE-RJ aponta sobrepreço na compra de respiradores Data: 25/05/2020 A auditoria foi realizada entre 27 e 30 de abril em três contratos assinados para a aquisição de mil respiradores, com valor total de R$ 183,5 milhões - Imprensa/RJ O Tribunal de Contas do Estado do Rio de Janeiro (TCE-RJ), por meio da Secretaria Geral de Controle Externo (SGE), concluiu o relatório de Auditoria Governamental para verificar irregularidades nos contratos emergenciais da Secretaria Estadual de Saúde do Rio para a aquisição de ventiladores pulmonares no combate à pandemia do novo coronavírus e apurou um sobrepreço de R$ 123 milhões na aquisição dos respiradores. A auditoria aponta o ex-secretário estadual de Saúde, Edmar José dos Santos e o ex-subsecretário Gabriell Carvalho Neves Franco dos Santos como principais responsáveis pelas irregularidades e solicita a apresentação de razões de defesa ou o ressarcimento aos cofres públicos de R$ 36,5 milhões. A auditoria foi realizada ent

In [ ]:
key = input("Key:")

In [385]:
import openai

openai.api_key = key

def openai_chat_completion_response(final_prompt, prompt=HEADER_PROMPT):
  response = openai.ChatCompletion.create(
              model="gpt-3.5-turbo",
              messages=[
                    {"role": "system", "content": prompt},
                    {"role": "user", "content": "Entendeu?"},
                    {"role": "assistant", "content": "Entendi sim!"},
                    {"role": "user", "content": final_prompt}
                ]
            )

  return response

In [ ]:
final_prompt = guidelines_prompt + ' '.join(full_data[0]["text"].split()[:max_length]) + "\nSaída: "

In [ ]:
response = openai_chat_completion_response(final_prompt)

Exemplos:
1. Sentença: TCE-RJ aponta sobrepreço na compra de respiradores Data: 25/05/2020 A auditoria foi realizada entre 27 e 30 de abril em três contratos assinados para a aquisição de mil respiradores, com valor total de R$ 183,5 milhões - Imprensa/RJ O Tribunal de Contas do Estado do Rio de Janeiro (TCE-RJ), por meio da Secretaria Geral de Controle Externo (SGE), concluiu o relatório de Auditoria Governamental para verificar irregularidades nos contratos emergenciais da Secretaria Estadual de Saúde do Rio para a aquisição de ventiladores pulmonares no combate à pandemia do novo coronavírus e apurou um sobrepreço de R$ 123 milhões na aquisição dos respiradores. A auditoria aponta o ex-secretário estadual de Saúde, Edmar José dos Santos e o ex-subsecretário Gabriell Carvalho Neves Franco dos Santos como principais responsáveis pelas irregularidades e solicita a apresentação de razões de defesa ou o ressarcimento aos cofres públicos de R$ 36,5 milhões. A auditoria foi realizada entre

In [ ]:
content = response["choices"][0]["message"]["content"]

In [ ]:
content = content.replace("'",'"')

In [ ]:
content = json.loads(content, object_hook=lambda d: defaultdict(int, d))

In [ ]:
def calc_metrics(resp_list, labels_list):
  total_target_entities = 0
  for labels in labels_list:
    for ent_type, entities in labels.items():
      total_target_entities += len(entities)

  #Calculates recall
  recall = 0
  for i, labels in enumerate(labels_list):
    for ent_type, entities in labels.items():
      for ent in entities:
        ent = ent.strip()
        if ent in resp_list[i][ent_type]:
          recall += 1

  recall = recall/total_target_entities

  #Calculates precision
  total_model_entities = 0
  for resp in resp_list:
    for ent_type, entities in resp.items():
      total_model_entities += len(entities)

  precision = 0
  for i, resp in enumerate(resp_list):
    for ent_type, entities in resp.items():
      for ent in entities:
        ent = ent.strip()
        if ent in labels_list[i][ent_type]:
          precision += 1

  precision = precision/total_model_entities

  f1 = 2 * (precision * recall) / (precision + recall)

  return recall, precision, f1
  

In [ ]:
resp_list = []
labels_list = []


In [374]:
start = 161
end = 187

In [ ]:
for i, item in enumerate(full_data[start:end]):
  text = item["text"]
  labels = item["labels"]
  final_prompt = guidelines_prompt + text + "\nSaída: "
  response = openai_chat_completion_response(final_prompt)
  content = response["choices"][0]["message"]["content"]
  content = content.replace("'",'"')
  content = json.loads(content, object_hook=lambda d: defaultdict(list, d))
  resp_list.append(content)
  labels_list.append(labels)

In [376]:
len(resp_list)

187

In [377]:
len(labels_list)

187

In [378]:
recall, precision, f1 = calc_metrics(resp_list, labels_list)
print("Recall = ", recall)
print("Precision = ", precision)
print("F1 = ", f1)

Recall =  0.6067663643049767
Precision =  0.7745302713987474
F1 =  0.6804605248044154


Salva os resultados para evitar chamadas adicionais à API

In [380]:
main_path

'/content/gdrive/MyDrive/Unicamp-aula-4'

In [382]:
import pickle

with open(f"{main_path}/resp_list_few_shot.pickle", 'wb') as f:
    pickle.dump(resp_list, f)


## Few-shot sem instrução no cabeçalho

In [387]:
resp_list = []
labels_list = []

In [409]:
start = 82
end = 187

In [ ]:
for i, item in enumerate(full_data[start:end]):
  text = item["text"]
  labels = item["labels"]
  final_prompt = guidelines_prompt + text + "\nSaída: "
  #Sem prompt no cabeçalho
  response = openai_chat_completion_response(final_prompt, prompt='')
  content = response["choices"][0]["message"]["content"]
  content = content.replace("'",'"')
  content = json.loads(content, object_hook=lambda d: defaultdict(list, d))
  resp_list.append(content)
  labels_list.append(labels)

In [411]:
len(resp_list)

187

In [412]:
len(labels_list)

187

In [413]:
recall, precision, f1 = calc_metrics(resp_list, labels_list)
print("Recall = ", recall)
print("Precision = ", precision)
print("F1 = ", f1)

Recall =  0.5741603334150527
Precision =  0.7095980311730927
F1 =  0.6347347809487763


In [414]:
with open(f"{main_path}/resp_list_few_shot_no_inst.pickle", 'wb') as f:
    pickle.dump(resp_list, f)

## Zero-shot learning

In [415]:
resp_list = []
labels_list = []

In [416]:
start = 0
end = 187

In [418]:
response = openai_chat_completion_response(full_data[0]["text"])

In [422]:
response["choices"][0]["message"]["content"]

'Entidades do tipo INSTITUIÇÃO PÚBLICA: Ministério Público do Estado de Mato Grosso (MPMT), Prefeitura Municipal de Rondonópolis. \n\nEntidades do tipo PESSOA: José Carlos Junqueira de Araújo (prefeito), Izalba Diva de Albuquerque (secretária municipal de Saúde), outros dois servidores públicos, empresários proprietários das empresas envolvidas no contrato para aquisição de 22 ventiladores pulmonares, Jesus de Oliveira Vieira de Souza (suposto proprietário da empresa Life Med Comércio de Produtos Hospitalares e Medicamentos Eireli), Ramos de Faria e Silva Filho (real proprietário da empresa Life Med Comércio de Produtos Hospitalares e Medicamentos Eireli).\n\nEntidades do tipo ORGANIZAÇÃO PRIVADA: empresas envolvidas no contrato para aquisição de 22 ventiladores pulmonares (Life Med Comércio de Produtos Hospitalares e Medicamentos Eireli e outras empresas cotadas na suposta cotação de preços apresentada). \n\nEntidades do tipo LOCAL: Não há informações suficientes na matéria para deter

In [425]:
def process_textual_content(content):
  map_ent_type_to_text = dict()

  for ent_type in entity_types_map.values():
    pattern = f"Entidades do tipo {ent_type}:"
    index = content.find(pattern)
    index += len(pattern)
    text = content[index:]
    text = text[:text.find("\n\nEntidades do tipo")]
    map_ent_type_to_text[ent_type] = text

  return map_ent_type_to_text


In [426]:
process_textual_content(response["choices"][0]["message"]["content"])

{'PESSOA': ' José Carlos Junqueira de Araújo (prefeito), Izalba Diva de Albuquerque (secretária municipal de Saúde), outros dois servidores públicos, empresários proprietários das empresas envolvidas no contrato para aquisição de 22 ventiladores pulmonares, Jesus de Oliveira Vieira de Souza (suposto proprietário da empresa Life Med Comércio de Produtos Hospitalares e Medicamentos Eireli), Ramos de Faria e Silva Filho (real proprietário da empresa Life Med Comércio de Produtos Hospitalares e Medicamentos Eireli).',
 'LOCAL': ' Não há informações suficientes na matéria para determinar se há entidades do tipo LOCAL envolvidas no caso',
 'ORGANIZAÇÃO PRIVADA': ' empresas envolvidas no contrato para aquisição de 22 ventiladores pulmonares (Life Med Comércio de Produtos Hospitalares e Medicamentos Eireli e outras empresas cotadas na suposta cotação de preços apresentada). ',
 'INSTITUIÇÃO PÚBLICA': ' Ministério Público do Estado de Mato Grosso (MPMT), Prefeitura Municipal de Rondonópolis. '}

Conforme pode ser visto pela saída acima, o ChatGPT não somente retorna entidades, como também contextualiza (define o papel de cada uma de acordo com o texto), uma funcionalidade impressionante.  

Por outro lado, não é possível medir facilmente a precisão da saída, uma vez que a resposta não aponta precisamente as entidades, deixando alguns nomes que de fato aparecem no texto em meio a expressões como *e outro(a)s...*.  Por esse motivo, aqui só será medida a revocação (recall).  Adicionalmente, será necessário fazer um parsing mínimo da saída para agrupar os textos pot tipo de entidade.

In [435]:
def calc_recall(resp_list, labels_list):
  total_target_entities = 0
  for labels in labels_list:
    for ent_type, entities in labels.items():
      total_target_entities += len(entities)

  #Calculates recall
  recall = 0
  for i, labels in enumerate(labels_list):
    for ent_type, entities in labels.items():
      for ent in entities:
        ent = ent.strip()
        if ent in resp_list[i][ent_type]:
          #print(ent)
          recall += 1

  recall = recall/total_target_entities

  return recall

In [432]:
full_data[0]["labels"]

defaultdict(list,
            {'PESSOA': ['José Carlos Junqueira de Araújo',
              'Izalba Diva de Albuquerque',
              'Marcos Brant',
              'Jesus de Oliveira Vieira de Souza',
              'Ramos de Faria e Silva Filho'],
             'ORGANIZAÇÃO PRIVADA': ['Life Med Comércio de Produtos Hospitalares e Mediamentos Eireli'],
             'INSTITUIÇÃO PÚBLICA': ['Ministério Público do Estado de Mato Grosso',
              'MPMT',
              'MPMT',
              'Prefeitura Municipal de Rondonópolis',
              '2ª Promotoria de Justiça Cível de Rondonópolis',
              'Município',
              '2ª Promotoria de Justiça Cível de Rondonópolis',
              'Município',
              'Ministério Público',
              'MPMT',
              'Justiça'],
             'LOCAL': ['Rondonópolis', 'Palmas', 'TO']})

In [431]:
calc_recall([process_textual_content(response["choices"][0]["message"]["content"])], [full_data[0]["labels"]])

José Carlos Junqueira de Araújo
Izalba Diva de Albuquerque
Jesus de Oliveira Vieira de Souza
Ramos de Faria e Silva Filho
Ministério Público do Estado de Mato Grosso
MPMT
MPMT
Prefeitura Municipal de Rondonópolis
Ministério Público
MPMT


0.5

Ao analisar com mais detalhes as diferenças entre o que foi retornado pelo modelo e o que era esperado, verificamos que a métrica foi em parte impactada por um detalhe surpreendente: a string *Life Med Comércio de Produtos Hospitalares e Mediamentos Eireli*, presente no texto original, contém um erro de ortografia ("Mediamentos").  Por outro lado, o ChatGPT retornou tal entidade, porém com a grafia devidamente corrigida para "Medicamentos". Por esse motivo, esse acerto não foi computado pelo cálculo simplificado da métrica recall feito acima.  Certamente o mesmo problema de estimativa das métricas para baixo ocorreu pelo mesmo motivo no few-shot learning (com e sem instrução) Portanto, conclui-se que, para medir com mais precisão a saída do modelo, faz-se necessário usar comparações de string mais flexíveis ("fuzzy").

In [ ]:
for i, item in enumerate(full_data[start:end]):
  print("Request: ", i)
  text = item["text"]
  labels = item["labels"]
  #Sem guidelines
  response = openai_chat_completion_response(text)
  content = response["choices"][0]["message"]["content"]
  resp_list.append(process_textual_content(content))
  labels_list.append(labels)

In [437]:
len(resp_list)

187

In [438]:
len(labels_list)

187

In [436]:
recall = calc_recall(resp_list, labels_list)
print("Recall = ", recall)


Recall =  0.481000245158127


## Conclusões

Os resultados seguiram um padrão previsto, de melhorarem com um número de exemplos (few-shot learning), preferencialmente com prompt de instrução.  Quando retiramos o prompt de instrução, notamos uma piora nas métricas, e essa piora (com relação à métrica recall) se torna ainda mais acentuada no modo zero-shot learning.  Em comparacão ao valor de 88% de F1 obtido com finetuning utilizando o mesmod dataset, o melhor valor para F1 obtido aqui foi de 68%, com few-shot learning.

Entretanto, vale ressaltar que, apesar da grande diferença numéricas entre as métricas, o modelo tem um comportamento qualitativo superior com relacão a dois pontos:
* correção ortográfica de *typos*
* explicação de contextualização de cada entidade (modo zero-shot): além de apontar as entidades, o modelo informa o papel de cada uma no contexto dado.